# Integración de Datos Externos: APIs Climáticas + Google Gemini

En esta lección aprenderemos a combinar datos climáticos en tiempo real con análisis de IA usando Google Gemini API, creando herramientas prácticas para la toma de decisiones agrícolas.

## Configuración de APIs: Clima + Google Gemini

Vamos a utilizar dos APIs:
1. **OpenWeatherMap**: Para datos climáticos en tiempo real
2. **Google Gemini**: Para análisis inteligente de los datos climáticos

La mayoría de las APIs requieren una clave de API (como una contraseña única).

In [ ]:
# Instalación de librerías
%pip install -U -q google-genai python-dotenv
import requests
import matplotlib.pyplot as plt
from IPython.display import Markdown

In [ ]:
# Configuración de Google Gemini API
from google.colab import userdata
import os
from google import genai
from google.genai import types
from dotenv import load_dotenv

load_dotenv()  # Cargar variables de entorno desde un archivo .env si existe

try:
    # En Colab
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
except:
    # En entorno local
    GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')

if not GOOGLE_API_KEY:
    raise ValueError('GOOGLE_API_KEY no encontrada. Configura tu clave de Google AI Studio')

# Inicializar cliente Gemini
cliente_gemini = genai.Client(api_key=GOOGLE_API_KEY)
MODEL_ID = "gemini-2.5-pro"

print("✅ Google Gemini API configurada correctamente")

In [ ]:
# API key de OpenWeatherMap (pública para el curso)
api_key = "93fd77ad60c515e9c1dce85692ef4529"

## Obtención de Datos Climáticos

Llamamos a la API del clima pasando las coordenadas y la clave de API:

In [ ]:
#url = f"http://api.openweathermap.org/data/2.5/weather?q=Buenos+Aires&appid={api_key}"

In [ ]:
url = f"http://api.openweathermap.org/data/2.5/weather?q=Mar+del+Plata&units=metric&appid={api_key}"

In [ ]:
# Obtener respuesta de la API climática
response = requests.get(url)
data = response.json()

print("Datos climáticos obtenidos:")
print(data)

## Procesamiento de Datos Climáticos

Extraemos la información relevante del JSON y la formateamos para análisis agrícola:

In [ ]:
# Extraer datos específicos
temperatura = data['main']['temp']
descripcion = data['weather'][0]['description']
velocidad_viento = data['wind']['speed']
humedad = data['main']['humidity']
presion = data['main']['pressure']

# Mostrar datos organizados
print("=== CONDICIONES CLIMÁTICAS ACTUALES ===")
print(f"Temperatura: {temperatura}°C")
print(f"Descripción: {descripcion}")
print(f"Velocidad del viento: {velocidad_viento} m/s")
print(f"Humedad relativa: {humedad}%")
print(f"Presión atmosférica: {presion} hPa")

In [ ]:
# Crear informe climático estructurado
informe_clima = f"""REPORTE CLIMÁTICO - REGIÓN PAMPEANA
Fecha: Tiempo actual
Ubicación: Buenos Aires, Argentina

CONDICIONES METEOROLÓGICAS:
- Temperatura: {temperatura}°C
- Condición general: {descripcion}
- Humedad relativa: {humedad}%
- Velocidad del viento: {velocidad_viento} m/s
- Presión atmosférica: {presion} hPa
"""

print(informe_clima)

## Análisis Agronómico con Google Gemini

Ahora usamos Gemini para analizar los datos climáticos y generar recomendaciones agrícolas específicas:

In [ ]:
# Función para consultar Gemini
def consultar_gemini(prompt):
    response = cliente_gemini.models.generate_content(
        model=MODEL_ID,
        contents=[prompt]
    )
    return response.text

# Análisis agronómico con Gemini
prompt_agricola = f"""Como agrónomo especialista en la región pampeana argentina, analiza las siguientes condiciones climáticas y proporciona recomendaciones técnicas específicas:

{informe_clima}

Incluye en tu análisis:
1. ¿Es un buen momento para aplicaciones de agroquímicos?
2. ¿Qué precauciones deben tomar los operarios agrícolas?
3. ¿Cómo afectan estas condiciones a los cultivos de soja y maíz?
4. ¿Se recomienda riego en estas condiciones?
5. Recomendaciones específicas para las próximas 24-48 horas

Responde de forma técnica pero práctica."""

respuesta_gemini = consultar_gemini(prompt_agricola)
print("=== ANÁLISIS AGRONÓMICO CON GEMINI ===")
display(Markdown(respuesta_gemini))

## Visualización de Datos Climáticos

Creamos gráficos para mejor comprensión de las condiciones actuales:

In [ ]:
# Crear gráfico de condiciones actuales
fig, ax = plt.subplots(figsize=(12, 6))

parametros = ['Temperatura (°C)', 'Humedad (%)', 'Viento (m/s)', 'Presión (hPa)']
valores = [temperatura, humedad, velocidad_viento, presion/10]  # Presión dividida por 10 para escala
colores = ['red', 'blue', 'green', 'orange']

bars = ax.bar(parametros, valores, color=colores)
ax.set_ylabel('Valor')
ax.set_title('Condiciones Climáticas Actuales - Buenos Aires')

# Agregar valores sobre las barras
for bar, valor in zip(bars, [temperatura, humedad, velocidad_viento, presion]):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height + 0.5,
            f'{valor:.1f}',
            ha='center', va='bottom')

plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

## Pronóstico Extendido para Planificación Agrícola

Obtenemos datos de varios días para planificación de actividades de campo:

In [ ]:
# Obtener pronóstico de 5 días
url_5dias = f"http://api.openweathermap.org/data/2.5/forecast?q=Rosario&units=metric&lang=es&appid={api_key}"

response_5dias = requests.get(url_5dias)
data_5dias = response_5dias.json()

# Extraer datos para análisis
fechas = []
temperaturas = []
humedades = []
velocidades_viento = []

for pronostico in data_5dias['list'][:20]:  # Primeros 20 registros (5 días)
    fechas.append(pronostico['dt_txt'])
    temperaturas.append(pronostico['main']['temp'])
    humedades.append(pronostico['main']['humidity'])
    velocidades_viento.append(pronostico['wind']['speed'])

# Crear gráfico de tendencia
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(15, 12))

# Temperatura
ax1.plot(fechas, temperaturas, 'r-o', linewidth=2)
ax1.set_title('Pronóstico de Temperatura (5 días)')
ax1.set_ylabel('Temperatura (°C)')
ax1.tick_params(axis='x', rotation=45)
ax1.grid(True, alpha=0.3)

# Humedad
ax2.plot(fechas, humedades, 'b-s', linewidth=2)
ax2.set_title('Pronóstico de Humedad Relativa (5 días)')
ax2.set_ylabel('Humedad (%)')
ax2.tick_params(axis='x', rotation=45)
ax2.grid(True, alpha=0.3)

# Velocidad del viento
ax3.plot(fechas, velocidades_viento, 'g-^', linewidth=2)
ax3.set_title('Pronóstico de Velocidad del Viento (5 días)')
ax3.set_ylabel('Velocidad (m/s)')
ax3.set_xlabel('Fecha y Hora')
ax3.tick_params(axis='x', rotation=45)
ax3.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Pronóstico obtenido para {len(fechas)} períodos de 3 horas")

## Análisis Predictivo con Gemini

Usamos los datos del pronóstico extendido para generar un plan de actividades agrícolas:

In [ ]:
# Preparar resumen del pronóstico para análisis
temp_promedio = sum(temperaturas) / len(temperaturas)
humedad_promedio = sum(humedades) / len(humedades)
viento_promedio = sum(velocidades_viento) / len(velocidades_viento)
temp_maxima = max(temperaturas)
temp_minima = min(temperaturas)

resumen_pronostico = f"""PRONÓSTICO EXTENDIDO (5 DÍAS) - PAMPA HÚMEDA

RESUMEN ESTADÍSTICO:
- Temperatura promedio: {temp_promedio:.1f}°C
- Temperatura máxima: {temp_maxima:.1f}°C
- Temperatura mínima: {temp_minima:.1f}°C
- Humedad relativa promedio: {humedad_promedio:.1f}%
- Velocidad de viento promedio: {viento_promedio:.1f} m/s

TENDENCIAS OBSERVADAS:
- Rango térmico: {temp_maxima - temp_minima:.1f}°C
- Condiciones de viento: {'Calmo' if viento_promedio < 3 else 'Moderado' if viento_promedio < 6 else 'Fuerte'}
- Nivel de humedad: {'Bajo' if humedad_promedio < 50 else 'Moderado' if humedad_promedio < 70 else 'Alto'}"""

print(resumen_pronostico)

In [ ]:
# Generar plan de actividades con Gemini
prompt_planificacion = f"""Como asesor agropecuario especializado en la región pampeana, analiza este pronóstico extendido y genera un PLAN DE ACTIVIDADES DE CAMPO para los próximos 5 días:

{resumen_pronostico}

CONSIDERA EN TU ANÁLISIS:
- Cultivos típicos de la zona: soja, maíz, trigo
- Actividades agrícolas: pulverizaciones, fertilización, siembra, cosecha
- Condiciones óptimas para cada actividad
- Restricciones por viento, humedad y temperatura
- Seguridad de los operarios

GENERA:
1. CRONOGRAMA DIARIO de actividades recomendadas
2. VENTANAS ÓPTIMAS para aplicaciones
3. ACTIVIDADES A EVITAR en cada período
4. ALERTAS ESPECIALES si las hubiera
5. PREPARATIVOS recomendados

Formato: Día por día con horarios específicos."""

plan_actividades = consultar_gemini(prompt_planificacion)
print("=== PLAN DE ACTIVIDADES AGRÍCOLAS (5 DÍAS) ===")
display(Markdown(plan_actividades))

## Comparación Regional con Análisis Multi-zona

Comparamos condiciones entre diferentes regiones agrícolas de Argentina:

In [ ]:
# Definir regiones agrícolas argentinas
regiones_agricolas = {
    "Pampa Húmeda": (-33.7, -60.5),
    "Cuyo": (-32.9, -68.8),
    "NOA (Salta)": (-26.8, -65.2),
    "NEA (Corrientes)": (-27.5, -58.8),
    "Patagonia Norte": (-41.8, -68.9)
}

def obtener_datos_clima_region(lat, lon):
    """Obtiene datos climáticos para una región específica"""
    url = f"https://api.openweathermap.org/data/2.5/weather?units=metric&lat={lat}&lon={lon}&appid={api_key}"
    try:
        response = requests.get(url)
        data = response.json()
        return {
            'temp': data['main']['temp'],
            'humedad': data['main']['humidity'],
            'viento': data['wind']['speed'],
            'descripcion': data['weather'][0]['description']
        }
    except Exception as e:
        print(f"Error obteniendo datos: {e}")
        return None

# Recopilar datos de todas las regiones
datos_regionales = {}
for region, coords in regiones_agricolas.items():
    print(f"Obteniendo datos para {region}...")
    datos = obtener_datos_clima_region(coords[0], coords[1])
    if datos:
        datos_regionales[region] = datos

print(f"\nDatos obtenidos para {len(datos_regionales)} regiones")

In [ ]:
# Visualizar comparación regional
if datos_regionales:
    fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(12, 15))

    regiones = list(datos_regionales.keys())
    temps = [datos['temp'] for datos in datos_regionales.values()]
    hums = [datos['humedad'] for datos in datos_regionales.values()]
    vientos = [datos['viento'] for datos in datos_regionales.values()]

    # Gráfico de temperaturas
    bars1 = ax1.bar(regiones, temps, color='red', alpha=0.7)
    ax1.set_title('Temperatura por Región Agrícola', fontsize=14, fontweight='bold')
    ax1.set_ylabel('Temperatura (°C)')
    ax1.tick_params(axis='x', rotation=45)

    # Agregar valores sobre barras
    for bar, temp in zip(bars1, temps):
        ax1.text(bar.get_x() + bar.get_width()/2., bar.get_height() + 0.3,
                f'{temp:.1f}°C', ha='center', va='bottom')

    # Gráfico de humedad
    bars2 = ax2.bar(regiones, hums, color='blue', alpha=0.7)
    ax2.set_title('Humedad Relativa por Región', fontsize=14, fontweight='bold')
    ax2.set_ylabel('Humedad (%)')
    ax2.tick_params(axis='x', rotation=45)

    for bar, hum in zip(bars2, hums):
        ax2.text(bar.get_x() + bar.get_width()/2., bar.get_height() + 1,
                f'{hum:.0f}%', ha='center', va='bottom')

    # Gráfico de viento
    bars3 = ax3.bar(regiones, vientos, color='green', alpha=0.7)
    ax3.set_title('Velocidad del Viento por Región', fontsize=14, fontweight='bold')
    ax3.set_ylabel('Velocidad del Viento (m/s)')
    ax3.set_xlabel('Regiones Agrícolas')
    ax3.tick_params(axis='x', rotation=45)

    for bar, viento in zip(bars3, vientos):
        ax3.text(bar.get_x() + bar.get_width()/2., bar.get_height() + 0.1,
                f'{viento:.1f}', ha='center', va='bottom')

    plt.tight_layout()
    plt.show()

    print("=== DATOS REGIONALES COMPARATIVOS ===")
    for region, datos in datos_regionales.items():
        print(f"{region}: {datos['temp']:.1f}°C, {datos['humedad']}% HR, {datos['viento']:.1f} m/s - {datos['descripcion']}")

## Análisis Regional Comparativo con Gemini

Usamos Gemini para generar recomendaciones específicas por región:

In [ ]:
# Análisis comparativo con Gemini
if datos_regionales:
    prompt_regional = f"""Como especialista en agricultura argentina, analiza las siguientes condiciones climáticas actuales en las principales regiones agrícolas del país:

{datos_regionales}

Proporciona un ANÁLISIS REGIONAL COMPARATIVO que incluya:

1. **CONDICIONES ÓPTIMAS**: ¿Qué región tiene las mejores condiciones hoy?
2. **CULTIVOS RECOMENDADOS**: ¿Qué cultivos son más apropiados para cada región según estas condiciones?
3. **ACTIVIDADES PRIORITARIAS**: ¿Qué actividades agrícolas se deberían realizar hoy en cada región?
4. **ALERTAS REGIONALES**: ¿Qué precauciones específicas debe tomar cada región?
5. **OPORTUNIDADES**: ¿Qué ventajas competitivas temporales tiene cada zona?

Considera:
- Cultivos típicos de cada región
- Épocas de siembra/cosecha por zona
- Sensibilidad climática de diferentes cultivos
- Logística y operaciones de campo

Respuesta técnica pero práctica para productores."""

    analisis_regional = consultar_gemini(prompt_regional)
    print("=== ANÁLISIS REGIONAL COMPARATIVO CON GEMINI ===")
    display(Markdown(analisis_regional))
else:
    print("No se pudieron obtener datos regionales para el análisis")

## Resumen y Próximos Pasos

### ¿Qué aprendimos?

1. **Integración de APIs**: Cómo combinar datos de múltiples fuentes (clima + IA)
2. **Google Gemini API**: Uso práctico para análisis agronómico en español
3. **Visualización de datos**: Gráficos informativos para toma de decisiones
4. **Análisis regional**: Comparación de condiciones entre zonas productivas
5. **Planificación agrícola**: Generación automática de cronogramas de actividades

### Aplicaciones Prácticas:
- Sistema de alertas climáticas para productores
- Planificación automática de pulverizaciones
- Optimización de ventanas de trabajo de campo
- Análisis comparativo regional para decisiones comerciales

### Próximo Cuaderno:
En **04-Automatización-Web-Scraping** aprenderemos a extraer datos de sitios web especializados y combinarlos con análisis de IA para crear sistemas más complejos.